In [1]:
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils

DATA_PATH = './data/'
TEST_PERCENT = 0.1
SELECT_SUBSET_PERCENT = 0.9

# The cat and dog images are of variable size we have to resize them to all the same size.
# DO NOT CHANGE
RESIZE_WIDTH=32
RESIZE_HEIGHT=32
# Setting to 5 epochs for it to run without GPU 
EPOCHS = 5

In [2]:
# loading the data.
X = []
Y = []

files = os.listdir(DATA_PATH)
# Shuffle so we are selecting about an equal number of dog and cat images.
shuffled_files = sklearn.utils.shuffle(files)
select_count = int(len(shuffled_files) * SELECT_SUBSET_PERCENT)

print('Going to load %i files' % select_count)

subset_files_select = shuffled_files[:select_count]

DISPLAY_COUNT = 1000

for i, input_file in enumerate(subset_files_select):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' % i)
        
    img = imread(DATA_PATH + input_file)
    # Resize the images to be the same size.
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    if 'cat' == input_file.split('.')[0]:
        Y.append(0.0)
    else:
        Y.append(1.0)
        
X = np.array(X)
Y = np.array(Y)

test_size = int(len(X) * TEST_PERCENT)

test_X = X[:test_size]
test_Y = Y[:test_size]
train_X = X[test_size:]
train_Y = Y[test_size:]

print('Train set has dimensionality %s' % str(train_X.shape))
print('Test set has dimensionality %s' % str(test_X.shape))

# Apply some normalization here.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255
test_X /= 255



Going to load 22500 files
Have loaded 1000 samples
Have loaded 2000 samples
Have loaded 3000 samples
Have loaded 4000 samples
Have loaded 5000 samples
Have loaded 6000 samples
Have loaded 7000 samples
Have loaded 8000 samples
Have loaded 9000 samples
Have loaded 10000 samples
Have loaded 11000 samples
Have loaded 12000 samples
Have loaded 13000 samples
Have loaded 14000 samples
Have loaded 15000 samples
Have loaded 16000 samples
Have loaded 17000 samples
Have loaded 18000 samples
Have loaded 19000 samples
Have loaded 20000 samples
Have loaded 21000 samples
Have loaded 22000 samples
Train set has dimensionality (20250, 32, 32, 3)
Test set has dimensionality (2250, 32, 32, 3)


In [3]:

# Import necessary layers.
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras import losses

model = Sequential()

# Define the network
model.add(Conv2D(600, input_shape=(32, 32, 3), kernel_size = 2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(300, kernel_size = 1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(150, kernel_size = 1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Define your loss and your objective
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

Using TensorFlow backend.


In [4]:
# Define the batch size
batch_size = 100

model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 16200 samples, validate on 4050 samples
Epoch 1/5
16200/16200 [==============================] - 390s - loss: 0.6719 - acc: 0.5788 - val_loss: 0.6185 - val_acc: 0.6768
Epoch 2/5
16200/16200 [==============================] - 353s - loss: 0.6210 - acc: 0.6540 - val_loss: 0.6229 - val_acc: 0.6496
Epoch 3/5
16200/16200 [==============================] - 325s - loss: 0.5921 - acc: 0.6852 - val_loss: 0.6113 - val_acc: 0.6546
Epoch 4/5
16200/16200 [==============================] - 375s - loss: 0.5623 - acc: 0.7102 - val_loss: 0.5206 - val_acc: 0.7449
Epoch 5/5
16200/16200 [==============================] - 388s - loss: 0.5388 - acc: 0.7246 - val_loss: 0.5180 - val_acc: 0.7437


In [5]:
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))

2250/2250 [==============================] - 9s     

Got 73.56% accuracy
